In [15]:
%run PullData.py

In [16]:
pd.options.display.max_rows = 300

In [17]:
MAIN_DATA = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/"
US_DATA = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports_us/"


In [18]:
Df_county = get_data(URL = MAIN_DATA,
              US_URL = US_DATA,
              Level = ['Country','FIPS'],
               switch_USA = False)

Df = get_data(URL = MAIN_DATA,
              US_URL = US_DATA,
              Level = ['Country'],
               switch_USA = True)

Pulling all data
skiping


/home/beltain/git/PythonPlaceHolder/Covid19/PullData.py:40: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  df = pd.concat([v for v in data_dict.values()], ignore_index=True)


Returning Data
Pulling all data
skiping
Pulling US Data
skiping
merging Data


/home/beltain/git/PythonPlaceHolder/Covid19/PullData.py:76: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  df = pd.concat([df, df_us])


In [19]:
# Loading FIps
#

Df_fips = pd.read_csv("./Data/fips_to_county.csv")
Df_fips['fipscounty'] = Df_fips.fipscounty.apply(lambda x: 44990 if x == 44999 else x)
Df_fips['fipscounty'] = Df_fips.fipscounty.apply(lambda x: x if x !=46113 else 46102)
Df_fips['fipscounty'] = Df_fips.fipscounty.apply(lambda x: x if x != 2270 else 2158 )
# Loading County/State level information
Df_state = pd.read_csv('./Data/co-est2019-alldata.csv', encoding = 'latin-1')\
                .rename(columns = {'POPESTIMATE2010':'pop',
                               'STATE':'state',
                               'COUNTY':'county'})
Df_state['county'] = Df_state.county.apply(lambda x: x if x > 0 else 990)

Df_state = Df_state\
            .assign(fipscounty = lambda x: x.state * 1000 + x.county)


Df_state = Df_state[['fipscounty','pop','state','county']]

#Loading world level infom
Df_wrld = pd.read_csv('./Data/WorldPopulation.csv') \
            .query("Type == 'Country/Area'")\
            .rename(columns = {'2020':'pop'})\
            [['Country','Country code','pop']]
    

In [20]:
Fips_pop = pd.merge(
                              Df_fips,
                              Df_state[['fipscounty','pop']],
                              how = 'inner',
                              on = ['fipscounty'])\
                              .assign(pop_100k = lambda x: round(x['pop']/100000,2))\
                              .rename(columns = {'fipsstate':'state_cd',
                                                 'fipscounty':'FIPS'}) \
                             .drop(columns = ['ssastate','ssacounty'])

In [21]:
Df_country_pop = pd.merge(Df_county \
             .assign(FIPS = lambda x: x.FIPS.astype(int)),
         Fips_pop,
         how = 'left',
         on = 'FIPS') \
    .fillna({'pop':-1,
             'pop_1ook':-1}) 

In [22]:
Df_country_pop.query("pop == -1").groupby('Country').Country.count()

Country
 Azerbaijan                             1
Afghanistan                           274
Albania                               260
Algeria                               273
Andorra                               267
Angola                                249
Antigua and Barbuda                   256
Argentina                             266
Armenia                               268
Aruba                                   7
Australia                             305
Austria                               273
Azerbaijan                            268
Bahamas                               247
Bahamas, The                            3
Bahrain                               274
Bangladesh                            261
Barbados                              252
Belarus                               270
Belgium                               294
Belize                                246
Benin                                 253
Bhutan                                263
Boats                     

In [14]:
pd.merge(Df_country_pop\
        [[]]
        )

TypeError: merge() missing 1 required positional argument: 'right'

In [23]:
#Bentoncount = 5007
#Duval, fl = 12031
#Washington, OK = 40147
#BAxter, tx = 48029
# Hamilton, OH = 39061
Df.query("FIPS in ['5007','12031','40147','48029','39061']")\
    .set_index('date')\
    .groupby("FIPS")\
    [['Active']]\
    .plot()

UndefinedVariableError: name 'FIPS' is not defined